# Evaluating Numerical Expressions
## Introduction

In this project, we will use stacks to implement an algorithm (similar to the `eval()` function) that can evaluate numerical expressions.

## Implementing Linked List and Stack Class

In [1]:
# Node implementation
class Node:
    def __init__(self, data):
        self.data = data
        self.prev = None
        self.next = None

In [2]:
# Linked List implementation
class LinkedList:

    def __init__(self):
        self.head = None
        self.tail = None
        self.length = 0

    def append(self, data):
        new_node = Node(data)
        if self.length == 0:
            self.head = self.tail = new_node
        else:
            self.tail.next = new_node
            new_node.prev = self.tail
            self.tail = new_node
        self.length += 1

    def __iter__(self):
        self._iter_node = self.head
        return self

    def __next__(self):
        if self._iter_node is None:
            raise StopIteration
        ret = self._iter_node.data
        self._iter_node = self._iter_node.next
        return ret

    def prepend(self, data):
        new_node = Node(data)
        if self.length == 0:
            self.head = self.tail = new_node
        else:
            self.head.prev = new_node
            new_node.next = self.head
            self.head = new_node
        self.length += 1

    def __len__(self):
        return self.length

    def __str__(self):
        return str([value for value in self])

In [3]:
# Stack implementation
class Stack(LinkedList):

    def push(self, data):
        self.append(data)

    def peek(self):
        return self.tail.data

    def pop(self):
        ret = self.tail.data
        if self.length == 1:
            self.tail = self.head = None
        else:
            self.tail = self.tail.prev
            self.tail.next = None
        self.length -= 1
        return ret

## Infix and Postfix Notation
Whenever we write an expression, we generally use infix notation. This means that put the operators between the operands. However, it is much easier for a computer to evaluate an expression in postfix notation. In this notation, the operands appear before the operator. For example, the infix expression `1 + 2` becomes `1 2 +` in postfix notation.

By using a stack, we can use the following steps to evaluate an expression written in postfix notation:
- Read the expression from left to right
- Push numbers to the top of the stack
- If we find an operator, pop the top two elements of the stack, perform the operation, and then push back the result

To start, we first need to separate each element of our expression string as tokens and then put them into a list.

In [4]:
def tokenize(expression):
    return expression.split()

In [5]:
# Test expression
expression = "12 / ( 2 + 4 ) * 21"
tokenize(expression)

['12', '/', '(', '2', '+', '4', ')', '*', '21']

## Processing an Operator
Whenever there is an operator token in our expression, we need to make sure the operation is being processed in the right order. To do this, we will write a function for each operator.

In [6]:
def process_minus(stack):
    second_element = stack.pop()
    first_element = stack.pop()
    stack.push(first_element - second_element)


def process_plus(stack):
    second_element = stack.pop()
    first_element = stack.pop()
    stack.push(first_element + second_element)


def process_times(stack):
    second_element = stack.pop()
    first_element = stack.pop()
    stack.push(first_element * second_element)


def process_divide(stack):
    second_element = stack.pop()
    first_element = stack.pop()
    stack.push(first_element / second_element)


def process_pow(stack):
    second_element = stack.pop()
    first_element = stack.pop()
    stack.push(first_element ** second_element)

## Evaluating Postfix Expressions
Here we will implement our algorithm to evaluate any expression in postfix notation.

In [7]:
def evaluate_postfix(expression):
    tokens = tokenize(expression)
    stack = Stack()

    for token in tokens:
        if token == '+':
            process_plus(stack)
        elif token == '-':
            process_minus(stack)
        elif token == '*':
            process_times(stack)
        elif token == '/':
            process_divide(stack)
        elif token == '**':
            process_pow(stack)
        else:
            stack.push(float(token))
    return stack.pop()

In [8]:
# Testing algorithm
expressions = [
    "4 6 -",
    "4 1 2 9 3 / * + 5 - *",
    "1 2 + 3 -",
    "1 2 - 3 +",
    "10 3 5 * 16 4 - / +",
    "5 3 4 2 - ** *",
    "12 2 4 + / 21 *",
    "1 1 + 2 **",
    "1 1 2 ** +"
]

for expression in expressions:
    print(evaluate_postfix(expression))

-2.0
8.0
0.0
2.0
11.25
45.0
42.0
4.0
2.0


## Operator Precedence in Infix Notation
In order to convert an expression from infix to postfix, we will implement the Shunting yard algorithm. To do this, we will first define a dictionary named `precedence` where the value of each key (operator) is the precedence value that determines the order in which the operators are evaluated.

In [9]:
precedence = {
    "+": 1,
    "-": 1,
    "*": 2,
    "/": 2,
    "**": 3
}

In [10]:
# Precedence tests
print(precedence["+"] < precedence["*"])
print(precedence["+"] < precedence["-"])
print(precedence["/"] < precedence["**"])

True
False
True


## From Infix to Postfix
Now we actually need to be able to convert our infix expressions into postfix expressions. In order to do this, we need to create functions that process each different token. Here is how each token will be processed:
- Opening parenthesis: Push the token into the stack for later use when we find a closing parenthesis.
- Closing parenthesis: While the top of the stack isn't an opening parenthesis, (, pop the top element, and append it to the postfix token list. After, pop the opening partheses out of the stack.
- Operator, `+`, `-`, `*`, `/`, or `**`: While the top of the stack is also an operator with a precedence greater than or equal to this operator, pop the top element, and append it to the postfix token list. Then push the current operator to the top of the stack.
- Operand (any number): Append the number to the `postfix` token list.

In [11]:
def process_opening_parenthesis(stack):
    stack.push("(")


def process_closing_parenthesis(stack, postfix):
    while stack.peek() != "(":
        postfix.append(stack.pop())
    stack.pop()


def process_operator(stack, postfix, operator):
    while len(stack) > 0 and stack.peek() in precedence and precedence[stack.peek()] >= precedence[operator]:
        postfix.append(stack.pop())
    stack.push(operator)


def process_number(postfix, number):
    postfix.append(number)

## Implementing the Shunting Yard Algorithm
Now we will finally implement the Shunting yard algorithm to process an expression from infix notation to postfix notation.

In [12]:
def infix_to_postfix(expression):
    tokens = tokenize(expression)
    stack = Stack()
    postfix = []
    for token in tokens:
        if token == "(":
            process_opening_parenthesis(stack)
        elif token == ")":
            process_closing_parenthesis(stack, postfix)
        elif token in precedence:
            process_operator(stack, postfix, token)
        else:
            process_number(postfix, token)

    while len(stack) > 0:
        postfix.append(stack.pop())

    return " ".join(postfix)

## Evaluating Infix Expressions
We can now combine the `infix_to_postfix()` and `evaluate_postfix()` functions into a single function that will return the result of the expression.

In [13]:
def evaluate(expression):
    postfix_expression = infix_to_postfix(expression)
    return evaluate_postfix(postfix_expression)

In [14]:
# Testing expressions
expressions = [
    "1 + 1",
    "1 * ( 2 - ( 1 + 1 ) )",
    "4 * ( 1 + 2 * ( 9 / 3 ) - 5 )",
    "10 + 3 * 5 / ( 16 - 4 * 1 )",
    "2 * 2 * 2 * 2 * 2 * 2 * 2 * 2",
    "2 ** 2 ** 2 ** 2 ** 2",
    "( 1 - 2 ) / ( 3 - 5 )",
    "9 / 8 * 8",
    "64 / ( 8 * 8 )",
]

for expression in expressions:
    print(evaluate(expression))

2.0
0.0
8.0
11.25
256.0
65536.0
0.5
9.0
1.0


## Conclusion: Final Thoughts
Now that we have successfully implemented an algorithm to evaluate expressions stored in strings, we must note that there are some limitations. For example, our algorithm requires us to input spaces inbetween each token in our expressions. Additionally, we could have also implemented other operators, such as the integer division operator `//`.